In [1]:
# Importing NumPy library for numerical computing in Python
import numpy as np

# Importing the 'os' library for operating system-related functions such as working with file paths
import os

# Importing the 'glob' library for pattern matching and file search
import glob

# Importing Matplotlib's pyplot library for creating plots
import matplotlib.pyplot as plt

In [2]:
def import_raw_files(filepath):
    """
    Read raw files from the given directory and import them into a list of 2D numpy arrays.
    
    Parameters:
        filepath (str): The directory path where the raw files are located.
        
    Returns:
        list: A list containing 2D numpy arrays, each corresponding to one raw file from the directory.
    """
    # Initialize an empty list to store the imported data
    raw_array = []

    # Use glob to get a list of filenames matching the pattern "*.raw" in the given filepath
    for filename in glob.glob(os.path.join(filepath, "*.raw")):
        # Open each raw file in read mode
        with open(filename, "r") as f:
            # Read the binary data from the file and convert it to a numpy array with data type uint16
            npimg = np.fromfile(f, dtype=np.uint16)
            
            # Reshape the 1-dimensional numpy array into a 2-dimensional array with 2048 columns (-1 for automatic calculation of rows)
            imp = np.reshape(npimg, (-1, 2048))
            
            # Append the reshaped array to the raw_array list
            raw_array.append(imp)
    
    return raw_array

In [3]:
def frame_plot_and_save(data_array, save_colorbar_path, save_plot_path, cmap='viridis', dpi=500):
    """
    Plot a 2D numpy array with logarithmic scale colormap, add a colorbar, and save the plot as an image.

    Parameters:
        data_array (numpy.ndarray): The 2D numpy array to plot.
        save_plot_path (str): The file path to save the main plot image.
        cmap (str, optional): The colormap to use for the plot. Default is 'viridis'.
        dpi (int, optional): The resolution (Dots Per Inch) of the saved images. Default is 500.

    Returns:
        None
    """
    # Add 1 to each element in the data array to avoid taking the logarithm of zero.
    data = np.asarray(data_array + 1)

    # Create a logarithmic normalization for the color map based on the data range
    log_norm = plt.matplotlib.colors.LogNorm(vmin=data.min(), vmax=data.max())

    # Create a new figure and axis for the plot
    fig, ax = plt.subplots()

    # Display the 2D array using the "imshow" function with the specified colormap and logarithmic normalization
    im = ax.imshow(data, cmap=cmap, norm=log_norm)

    # Add a colorbar to the plot using the previously defined normalization
    cbar = plt.colorbar(im)

    # Label the colorbar with a suitable description
    cbar.set_label('Logarithmic Scale')

    # Save the main plot as 'save_plot_path' with the specified resolution
    # The 'bbox_inches="tight"' and 'pad_inches=0' options are used to remove extra white space around the plot
    plt.savefig(save_plot_path, dpi=dpi, bbox_inches="tight", pad_inches=0)

    # Close the figure to release resources
    plt.close()

In [4]:
def sum_2d_arrays(raw_array):
    """
    Sum a list of 2D numpy arrays element-wise and return the final sum as a single 2D numpy array.

    Parameters:
        raw_array (list): A list containing 2D numpy arrays.

    Returns:
        numpy.ndarray: The sum of all 2D arrays in the list.
    """
    # Convert the list of 2D arrays into a 3D numpy array
    stacked_array = np.stack(raw_array, axis=0)

    # Sum all the 2D arrays along the first axis (axis=0) to get the final sum
    sum_exposures = np.sum(stacked_array, axis=0)

    return sum_exposures

In [5]:
def mean_2d_arrays(raw_array):
    """
    Calculate the mean of a list of 2D numpy arrays element-wise and return the result as a single 2D numpy array.

    Parameters:
        raw_array (list): A list containing 2D numpy arrays.

    Returns:
        numpy.ndarray: The mean of all 2D arrays in the list.
    """
    # Convert the list of 2D arrays into a 3D numpy array
    stacked_array = np.stack(raw_array, axis=0)

    # Calculate the element-wise sum of all 2D arrays along the first axis (axis=0)
    sum_exposures = np.sum(stacked_array, axis=0)

    # Calculate the mean by dividing the sum by the number of arrays
    mean_exposures = sum_exposures / len(raw_array)

    return mean_exposures

In [6]:
def flatten_2d_array(input_array):
    """
    Flatten a 2D numpy array into a 1D numpy array.

    Parameters:
        input_array (numpy.ndarray): The 2D numpy array to be flattened.

    Returns:
        numpy.ndarray: The flattened 1D numpy array.
    """
    # Convert the 2D numpy array into a 1D numpy array using np.ravel()
    flattened_array = np.ravel(input_array)
    return flattened_array

In [7]:
def plot_histogram(data_array, filename, xlim=None, dpi=None, color=None):
    """
    Plot a histogram of the given 1D numpy array, and save the plot as an image.

    Parameters:
        data_array (numpy.ndarray): The 1D numpy array to plot the histogram for.
        filename (str): The file path to save the histogram image.
        xlim (tuple, optional): The x-axis limits for the plot. Default is None (no specific limits).
        dpi (int, optional): The resolution (Dots Per Inch) of the saved image. Default is None (default DPI).
        color (str, optional): The color of the histogram bars. Default is None (default color).

    Returns:
        None
    """
    # Calculate the number of bins for the histogram based on the data range
    # The number of bins at 1 integer spacing is determined by the range of values in the data_array
    num_bins = np.int(data_array.max() + np.abs(data_array.min()))

    # Create the histogram plot using plt.hist()
    if color is not None:
        plt.hist(data_array, bins=num_bins, color=color)
    else:
        plt.hist(data_array, bins=num_bins)

    # Set the x-axis limits if provided
    if xlim is not None:
        plt.xlim(xlim)

    # Save the histogram plot as an image with the given filename and DPI (if provided)
    # The 'bbox_inches="tight"' and 'pad_inches=0' options are used to remove extra white space around the plot
    plt.savefig(filename, dpi=dpi, bbox_inches="tight", pad_inches=0)

    # Close the plot to release resources
    plt.close()

In [8]:
def subtract_array(subtract_this_array, from_this_array):
    """
    Subtract array from another array element-wise.

    Parameters:
        subtract_this_array (numpy.ndarray): The 2D numpy array representing the array to subtract.
        from_this_array (numpy.ndarray): The 2D numpy array representing the array we subtract from.

    Returns:
        numpy.ndarray: The result of subtracting the mean dark from the mean exposure array.
    """

    # Subtract 2D array subtract_this_array from  2D array from_this_array
    # This performs element-wise subtraction on corresponding elements of the arrays
    result_array = from_this_array - subtract_this_array

    return result_array



In [9]:
def plot_histogram_with_line(data_array, title, filename, xlim=None, xlabel="Data Number", ylabel="Frequency"):
    """
    Plot a histogram of the given 1D numpy array and overlay a line plot of the histogram data.

    Parameters:
        data_array (numpy.ndarray): The 1D numpy array to plot the histogram and line for.
        title (str): The title for the plot.
        filename (str): The file path to save the plot image.
        xlim (tuple, optional): The x-axis limits for the plot. Default is None (no specific limits).
        xlabel (str, optional): The label for the x-axis. Default is "Data Number".
        ylabel (str, optional): The label for the y-axis. Default is "Frequency".

    Returns:
        None
    """
    # Calculate the total value range for the histogram bins
    totval = data_array.max() - data_array.min()

    # Calculate the number of bins for the histogram based on the total value range
    num_bins = np.int(np.ceil(totval))

    # Create the histogram using np.histogram()
    y, binEdges = np.histogram(data_array, bins=num_bins)

    # Calculate the bin centers for the line plot
    bincenters = 0.5 * (binEdges[1:] + binEdges[:-1])

    # Create the histogram plot using plt.plot()
    plt.plot(bincenters, y, '-', c='black')

    # Set the x-axis limits if provided
    if xlim is not None:
        plt.xlim(xlim)

    # Set the title and labels for the plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    # Save the plot as an image with the given filename
    # The 'bbox_inches="tight"' and 'pad_inches=0' options are used to remove extra white space around the plot
    plt.savefig(filename, bbox_inches="tight", pad_inches=0)

    # Close the plot to release resources
    plt.close()
